In [1]:
import pandas as pd
import numpy as np
import scipy

import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from lightgbm import LGBMRegressor
import lightgbm
import optuna
import warnings
from tqdm import TqdmExperimentalWarning
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import shap

c:\Users\David\Projets\Energetic-performance-explainability\venvexpl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lecture des données

In [2]:


file_path = "data/dataprocess.csv"

data = pd.read_csv(file_path,sep=",",decimal=".")

data['DateProd'] = data['DateProd'].apply(lambda x: datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S"))
data.set_index('DateProd',inplace=True)

In [10]:
data.head()

,conso_per_kg,type,var_quality,var_process,var_dim1,var_dim2
DateProd,,,,,,
2025-01-01 00:00:00,1743.7111,prod_0,1090.0,44.868355,1031.0112,60.306684
2025-01-01 00:06:00,1622.7773,prod_1,1090.0,46.858467,1038.0148,60.504509
2025-01-01 00:12:00,1604.9292,prod_2,1090.0,46.823181,1034.7076,60.452483
2025-01-01 00:18:00,1595.9277,prod_2,1090.0,45.785667,1032.9819,60.655019
2025-01-01 00:24:00,1709.8407,prod_3,1090.0,44.991562,1034.5690,60.273794


In [5]:
data.to_parquet('data/dataprocess.parquet')

In [3]:
features = ['type','var_quality','var_process','var_dim1','var_dim2']
target   = 'conso_per_kg'

data = data[[target]+features]

## Suppression des outliers

In [4]:
cols = [target, 'var_quality','var_process','var_dim1','var_dim2']

# Création d'une grille 2 lignes × 3 colonnes
fig = make_subplots(rows=2, cols=3, subplot_titles=cols)

# Ajout des boxplots
for i, col in enumerate(cols):
    row = i // 3 + 1
    col_pos = i % 3 + 1
    fig.add_trace(go.Box(y=data[col], name=col), row=row, col=col_pos)

fig.update_layout(
    height=600, width=1000,
    title_text="Boxplots des 5 variables (avant suppression outliers)",
    showlegend=False
)

fig.show()


![Box plot avant suppression outliers](images/box_plot_before_cleaning.png)

In [4]:
data = data[data.conso_per_kg>0] # On ne s'intéresse qu'aux consommations > 0
data = data[data.var_process>30]
data = data[data.var_dim1>500]
data = data[data.var_dim2>30]

In [ ]:
# Création d'une grille 2 lignes × 3 colonnes
fig = make_subplots(rows=2, cols=3, subplot_titles=cols)

# Ajout des boxplots
for i, col in enumerate(cols):
    row = i // 3 + 1
    col_pos = i % 3 + 1
    fig.add_trace(go.Box(y=data[col], name=col), row=row, col=col_pos)

fig.update_layout(
    height=600, width=1000,
    title_text="Boxplots des 5 variables (après suppression outliers)",
    showlegend=False
)

fig.show()

![Box plot avant suppression outliers](images/box_plot_after_cleaning.png)

## Construction du preprocesseur

In [6]:
def BuildPrepro(data:pd.DataFrame, features:list):

    """
    Function that build the preprocessor of the model pipeline
    Taking into account possible categorical variables

    """

    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import StandardScaler
    import numpy as np
    
    num_feat = [f for f in data[features].columns if data.dtypes[f]==np.float64]
    cat_feat = [f for f in data[features].columns if data.dtypes[f]==object]

    if len(cat_feat)>0:
        num_prepro  = StandardScaler()
        cat_prepro  = OneHotEncoder(handle_unknown='ignore')
        prepro =  ColumnTransformer([('num',num_prepro,num_feat),('cat',cat_prepro, cat_feat)])
    else:
        num_prepro = StandardScaler()
        prepro =  ColumnTransformer([('num',num_prepro,num_feat)])

    return prepro

## Réduction de dimensionnalité

In [7]:
nb_type = len(data.type.unique())
print(f"nombre de catégories {nb_type}")

nombre de catégories 90


Le nombre de catégories est assez élevé. On effectue une réduction de dimensionnalité afin de ne conserver que les catégories les plus influentes.
Ici, j'utilise la propriété feature_importance des arbres de régression pour classer le poids des catégories

In [8]:
y = data[target]
X = data[features]
prepro = BuildPrepro(data = data, features = features)
X_onehot = prepro.fit_transform(X)
model_reduction =DecisionTreeRegressor()
model_reduction.fit(X_onehot, y)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [9]:
df_feat_importance = pd.DataFrame(data={"features":prepro.get_feature_names_out().tolist(), # dataframe avec le poids obtenus pour chaque features
                                        "importance":model_reduction.feature_importances_})


In [55]:
df_feat_importance.head(7)

,features,importance
0,num__var_quality,0.086154
1,num__var_process,0.093782
2,num__var_dim1,0.682526
3,num__var_dim2,0.054273
4,cat__type_prod_0,0.001013
5,cat__type_prod_1,0.004810
6,cat__type_prod_10,0.000505


In [ ]:
# On ne regarde que les features qui sont associées à un produit
df_feat_importance_prod = df_feat_importance[df_feat_importance['features'].str.contains('prod')]
df_feat_importance_prod.sort_values(by='importance',ascending=False,inplace=True)
df_feat_importance_prod['perc'] = 100*df_feat_importance_prod['importance']/df_feat_importance_prod['importance'].sum()
df_feat_importance_prod['percum'] = df_feat_importance_prod['perc'].cumsum()

# on garde les catégories dont le cumul de l'importance est < 80 %
categorie_conservees = df_feat_importance_prod[df_feat_importance_prod.percum<80]['features'].to_list()

# on supprime la chaine "cat__type_" ajoutée par le transformeur
categorie_conservees =  [c.replace("cat__type_", "") for c in categorie_conservees] 

# On crée un dictionnaire pour mapper les types de produits les plus influents dans le dataframe "data"

dico_map_prod = {
    p: p if p in categorie_conservees else 'autres'
    for p in data.type.unique()
}

data['type'] = data['type'].map(dico_map_prod)

## Construction du modèle
Ici in utilise un lightgbm

In [11]:
def objective(trial,X_trs,Y):

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import root_mean_squared_error
    import numpy as np
    import warnings
    warnings.simplefilter('ignore')

    train_x, test_x, train_y, test_y = train_test_split(X_trs, Y, test_size=0.2,random_state=42)

    callbacks = [lightgbm.early_stopping(100, verbose=0), lightgbm.log_evaluation(period=0)]

    model = LGBMRegressor(verbosity = -1)

    
    param = { 
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5,0.6,0.7]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.02,0.04,0.08,0.12]),
        'max_depth': trial.suggest_categorical('max_depth', [4,5,6]),
        'n_estimators':trial.suggest_int('n_estimators',200,500,10),
        'num_leaves' : trial.suggest_int('num_leaves',100,200,20),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_categorical('subsample', [0.7,0.8,0.9])
    }

    fixed_hp =   {
            'metric': 'rmse', 
            'random_state': 48,
            'verbose': -1
        }

    for p, pv in fixed_hp.items():
        param[p] = pv

    model = LGBMRegressor(**param)

    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],callbacks=callbacks)

    preds_train = model.predict(train_x)  
    rmse_train = root_mean_squared_error(train_y, preds_train)
    preds_test = model.predict(test_x)
    rmse_test = root_mean_squared_error(test_y, preds_test)

    alpha_overfit = 0.4
    score_final = alpha_overfit*rmse_train + (1-alpha_overfit)*np.abs(rmse_train-rmse_test)
    
    return score_final

In [12]:
def FindHyperParams(data:pd.DataFrame, target:str, features:list):


    optuna.logging.set_verbosity(optuna.logging.INFO)
    warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)


    data.dropna(inplace=True)
    Y  = data[target]
    X  = data[features]

    prepro = BuildPrepro(data = data, features = features)

    X_trs = prepro.fit_transform(X)
    
    study = optuna.create_study(direction='minimize')

    Ntrial = 200


    study.optimize(lambda trial: objective(trial, X_trs, Y), n_trials = Ntrial)
    best_params = study.best_trial.params

    return best_params


In [13]:
best_params = FindHyperParams(data=data,target=target,features=features)

[I 2025-10-10 14:17:13,265] A new study created in memory with name: no-name-3559e7e2-c2c2-4588-a3cc-31fb01352e7d
[I 2025-10-10 14:17:17,331] Trial 0 finished with value: 132.65722055948584 and parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.12, 'max_depth': 6, 'n_estimators': 300, 'num_leaves': 120, 'reg_alpha': 0.1415435909757495, 'reg_lambda': 0.0016372476421746454, 'subsample': 0.7}. Best is trial 0 with value: 132.65722055948584.
[I 2025-10-10 14:17:18,229] Trial 1 finished with value: 137.23171202913426 and parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.02, 'max_depth': 6, 'n_estimators': 230, 'num_leaves': 180, 'reg_alpha': 0.008037335451060721, 'reg_lambda': 0.0039732657968850945, 'subsample': 0.7}. Best is trial 0 with value: 132.65722055948584.
[I 2025-10-10 14:17:19,103] Trial 2 finished with value: 131.66084182303737 and parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.12, 'max_depth': 4, 'n_estimators': 460, 'num_leaves': 200, 'reg_alpha': 0.0129

In [14]:
ml_pipeline = Pipeline([("preprocessor",prepro),("model", LGBMRegressor(**best_params))])
Y  = data[target]
X  = data[features]
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2,random_state=42)

ml_pipeline.fit(train_x,train_y)

preds_train = ml_pipeline.predict(train_x)  
preds_test  = ml_pipeline.predict(test_x)  
r2_train = r2_score(train_y,preds_train)
r2_test = r2_score(test_y,preds_test)

In [15]:
print(f"R2 train {r2_train} ; R2 test {r2_test}")

R2 train 0.8405725722607451 ; R2 test 0.816246645111479


In [7]:
import pandas as pd

dico_input= {'type': 'prod_1',
             'var_quality':1080,
             'var_process':45.5,
              'var_dim1':1032.5,
             'var_dim2':60.5}

df_input = pd.DataFrame(dico_input, index=[0])

In [8]:
df_input

,type,var_quality,var_process,var_dim1,var_dim2
0,prod_1,1080,45.5,1032.5,60.5


In [ ]:
type	var_quality	var_process	var_dim1	var_dim2
				
2025-01-01 00:00:00	1743.7111	prod_0	1090.0	44.868355	1031.0112	60.306684
2025-01-01 00:06:00	1622.7773	prod_1	1090.0	46.858467	1038.0148	60.504509
2025-01-01 00:12:00	1604.9292	prod_2	1090.0	46.823181	1034.7076	60.452483
2025-01-01 00:18:00	1595.9277	prod_2	1090.0	45.785667	1032.9819	60.655019
2025-01-01 00:24:00	1709.8407	prod_3	1090.0	44.991562	1034.5690	60.273794


NameError: name 'ml_pipeline' is not defined

A small overfitting but still very acceptable

In [16]:
## Enregistrement du modèle
import joblib
joblib.dump(ml_pipeline, 'models/model_energy_consumption.pkl')

['models/model_energy_consumption.pkl']

## Explicabilité avec shapeley

### Calcul des valeurs du shapeley

In [ ]:
import sklearn

def ComputeShapeleyValues(data:pd.DataFrame, 
                          features:str,
                          target:str,
                          model:sklearn.pipeline.Pipeline):
    """
    Calcule les valeurs du shapeley.

    Args:
        data (pandas dataframe): les données 
        feature: liste des facteur d'entrée
        target: nom de la variable modélisée
        model: pipeline sklearn (preporocesseur + regresseur)

    Returns:
        svals: valeurs du shapeley (dimension = [len(data), nbre features numérique + nbre de modalités de la variable catégorielle])
        df_x : pandas dataframe avec les features transformées même dimension que svals  

    """    
    # data: pandas datafra


    train_x = data[features]
    train_y = data[target]


    x_transformed = model.steps[0][1].transform(train_x)    # Ici je transforme "train_x" car shap ne peut pas intégrer directement des variables catégorielles
    if type(x_transformed) == scipy.sparse._csr.csr_matrix:
        x_transformed = np.asarray(x_transformed.todense())

    # Quelques manipulations pour récupérer les noms initiaux des features (qui ont été renommés par le transformeur)
    f_name = model.steps[0][1].get_feature_names_out()
    f_name = [f.replace('num__','').replace('cat__','') for f in f_name]

    regressor = model.steps[1][1] # on récupère le regresseur depuis le pipeline


    df_x = pd.DataFrame(index=train_x.index,data=x_transformed,columns=f_name) # pandas dataframe avec les features transformées


    # On crée un objet "explicateur" avec la librairie SHAP
    # On choisi un "TreeExplainer" qui est spécifiquement dédié (plus rapide) à l'explicabilité des modèles ensemblistes comme ce lgbm

    explainer = shap.TreeExplainer(regressor, 
                                   model_output='raw', 
                                   feature_perturbation='interventional')
    
    #model_output='raw' : SHAP calcule les contributions expliquant la valeur brute prédite par le modèle (avant transformation en probabilité).
    #feature_perturbation : définit comment SHAP simule l’effet de chaque variable lorsqu’on "perturbe" les entrées pour mesurer leur influence sur la prédiction.
    #feature_perturbation='interventional': C’est la version la plus rigoureuse : SHAP prend en compte les dépendances entre variables (corrélations).

    svals = explainer.shap_values(df_x, y=train_y)

    return svals, df_x




In [64]:
svals, df_x = ComputeShapeleyValues(data=data, 
                          features=features,
                          target=target,
                          model=ml_pipeline)

### Visualisation

In [ ]:
fig_shape_fact = plt.figure(facecolor="#c6dee8")
ax_shape = fig_shape_fact.axes
shap.summary_plot(svals, df_x,show=False)
plt.gca().tick_params(labelsize=10)
plt.gca().set_xlabel("Impact sur la variable modélisée", fontsize=14)

Le diagramme ci-dessous combine à la fois :

- la force d’impact de chaque variable sur le modèle,
- la direction de cet impact (positif ou négatif),
- la valeur des features (haute ou basse).

Lecture de l’axe horizontal

L’axe des X représente la valeur SHAP (l'unité est la même que la variable modélisée kwh/kg): c’est l’impact d’une variable sur la prédiction finale du modèle.
- Valeur SHAP positive → la variable augmente la prédiction (pousse vers une valeur plus élevée de la variable modélisée).
- Valeur SHAP négative → la variable diminue la prédiction.
- Zéro → pas d’impact sur la sortie du modèle pour cet échantillon.

Lecture de l’axe vertical

Chaque ligne correspond à une variable du modèle (feature). Elles sont triées par importance moyenne (du haut vers le bas).
- var_dim1 et var_quality sont les plus influentes,
- type_prod_40, type_prod_63, etc. ont un impact marginal.

Interprétation des couleurs

Les couleurs indiquent la valeur de la feature pour chaque observation :
- Si les points rouges sont surtout à droite → une valeur élevée augmente la sortie.
- Si les points rouges sont à gauche → une valeur élevée la diminue.


Par exemple pour la variable "var_quality":
- les points rouges sont plutôt à droite: une augmentation de la variable fait augmenter la consommation
- Moins étendue que "var_dim1": influence un peu plus faible que "var_dim1"

![Explicabilité détaillé des features](images/features_importance_detail.png)

### Exploitation pour expliquer les variations de consommations entre 2 mois

In [ ]:
df_shape = pd.DataFrame(data=svals,columns=df_x.columns)
df_shape.index = df_x.index
df_conso_mois = data[target].resample('MS').mean()
cols_types = [c for c in df_shape.columns if 'type_prod' in c]
cols_types.append('type_autres')
df_shape_type_aggrege = df_shape.copy()
df_shape_type_aggrege['type'] = df_shape_type_aggrege[cols_types].sum(axis=1)
df_shape_type_aggrege.drop(columns=cols_types,inplace=True)
df_shape_type_aggrege_mois = df_shape_type_aggrege.resample('MS').mean()

current_month  = "2025-07-01"
previous_month = "2025-06-01"

contributions_a_variation = df_shape_type_aggrege_mois.loc[current_month] - df_shape_type_aggrege_mois.loc[previous_month] 
conso_current_month       = df_conso_mois[current_month]
conso_previous_month      = df_conso_mois[previous_month]
part_unexplained          = conso_current_month - conso_previous_month - contributions_a_variation.sum()



x = ["Mois précédent"]     + contributions_a_variation.index.to_list() + ["Autre"] + ["Mois courant"]
y = [conso_previous_month] + contributions_a_variation.values.tolist() + [part_unexplained] +[conso_current_month]

fig = go.Figure(go.Waterfall(
    name="20",
    orientation="v",
    measure=["relative", "relative", "relative", "relative", "relative", "relative", "relative","total"],
    x=x,
    y=y,
    textposition="outside",
    connector={"line":{"color":"rgb(63, 63, 63)"}}
))

# Mise en forme
fig.update_layout(
      width=1000,      # Largeur en pixels
    height=600,     # Hauteur en pixels
    title="Variation de la consommation entre M-1 et M",
    xaxis_title="Facteur",
    yaxis_title="Consommation (kWh/kg)",
    yaxis=dict(range=[2000, 2800]),
    showlegend=False,
)

fig.show()

![Explication variation conso](images/waterfall_explanation.png)